In [ ]:
%env AWS-PROFILE=just-my-links

import boto3
import requests
import json


In [2]:
cloudformation_outputs_res = !aws cloudformation describe-stacks --stack-name just-my-links-dev --query 'Stacks[0].Outputs'
cloudformation_outputs = {x['OutputKey']: x['OutputValue'] for x in json.loads(''.join(cloudformation_outputs_res))}
application_bucket_name = cloudformation_outputs['ApplicationBucketName']
index_service_uri = cloudformation_outputs['HttpApiUrl']
index_service_arn = cloudformation_outputs['LambdaFunctionArn']
auth_token_secret_arn = cloudformation_outputs['BearerTokenSecretName']

See the logs

In [95]:
!aws logs tail /aws/lambda/dev-just-my-links-index-document

2025-06-05T23:13:57.768000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f START RequestId: ab2287e1-0d6c-4e8f-8da2-ab2e581cf391 Version: $LATEST
2025-06-05T23:13:57.769000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f {"level":"INFO","location":"lambda_handler:110","message":"Lambda handler invoked","timestamp":"2025-06-05 23:13:57,769+0000","service":"service_undefined","cold_start":true,"function_name":"dev-just-my-links-index-document","function_memory_size":"1024","function_arn":"arn:aws:lambda:us-east-1:145216492484:function:dev-just-my-links-index-document","function_request_id":"ab2287e1-0d6c-4e8f-8da2-ab2e581cf391","correlation_id":"LtqsFi-yIAMEZ1w=","event_type":"unknown","xray_trace_id":"1-684224b3-6315e57a476fb72c3d301224"}
2025-06-05T23:13:57.770000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f {"_aws":{"Timestamp":1749165237769,"CloudWatchMetrics":[{"Namespace":"just-my-links","Dimensions":[[]],"Metrics":[{"Name":"UnauthorizedRequests"

In [4]:
secrets_client = boto3.client('secretsmanager')
auth_token_response = secrets_client.get_secret_value(SecretId=auth_token_secret_arn)
auth_token = auth_token_response['SecretString']

In [8]:
response = requests.post(
    index_service_uri,
    headers={"Content-Type": "application/json", "Authorization": f"Bearer {auth_token}"},
    json={"url": "https://example.com", "title": "Test Document"}
)

response.status_code, response.text

(501,
 '{"message":"Not yet implemented - will download ChromaDB from S3, operate locally, then upload back"}')

Deploy changes to the lambda image with index-service deployment script

In [ ]:
! scripts/deploy-indexing-service.py dev

Turn on debug logging with

In [ ]:
function_name = 'dev-just-my-links-index-document'
lambda_client = boto3.client('lambda')

response = lambda_client.get_function_configuration(FunctionName=function_name)

current_variables = {**response.get('Environment', {}).get('Variables', {}), "LOG_LEVEL": "DEBUG"}

lambda_client.update_function_configuration(FunctionName=function_name, Environment={'Variables': current_variables})